In [9]:
# from pyspark import SparkContext, SparkConf
# from pyspark.sql import SparkSession

# try: 
#     sc.stop()
# except:
#     print("sc have not yet created!")
# conf = SparkConf().setAppName("First App").setMaster("spark://master:7077")
# sc = SparkContext(conf=conf)
# print(sc)
# my_spark = SparkSession(sc)

sc have not yet created!
<SparkContext master=spark://master:7077 appName=First App>


# Create Seasion

In [1]:
# khoi tao seasion
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

my_spark = SparkSession \
    .builder \
    .master("spark://linux:7077")\
    .appName("Firt Session") \
    .config("spark.some.config.option", "some-value") \
    .config("spark.executor.memory", "2g") \
    .config('spark.cores.max','4')\
    .getOrCreate()
my_spark

In [12]:
# show all data in my_spark 
my_spark.catalog.listTables()

[]

# Mở file lưu trên HDFS

In [3]:
Flights = my_spark.read.csv('hdfs://localhost:9001/user/input/flights.csv', header=True)
Airport = my_spark.read.csv('hdfs://localhost:9001/user/input/airports.csv', header=True)
Airline = my_spark.read.csv('hdfs://localhost:9001/user/input/airlines.csv', header=True)

In [4]:
Flights.show(5)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [5]:
# show info schema
Flights.printSchema()

root
 |-- YEAR: string (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- DAY: string (nullable = true)
 |-- DAY_OF_WEEK: string (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: string (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: string (nullable = true)
 |-- DEPARTURE_TIME: string (nullable = true)
 |-- DEPARTURE_DELAY: string (nullable = true)
 |-- TAXI_OUT: string (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- SCHEDULED_TIME: string (nullable = true)
 |-- ELAPSED_TIME: string (nullable = true)
 |-- AIR_TIME: string (nullable = true)
 |-- DISTANCE: string (nullable = true)
 |-- WHEELS_ON: string (nullable = true)
 |-- TAXI_IN: string (nullable = true)
 |-- SCHEDULED_ARRIVAL: string (nullable = true)
 |-- ARRIVAL_TIME: string (nullable = true)
 |-- ARRIVAL_DELAY: string (nullable = true)
 |-- D

# Thao tac SQL tren Dataframe

#### Đếm số lượng các chuyến bay trong tuần

In [8]:
%%time
Flights.groupBy('DAY_OF_WEEK')\
    .count()\
    .sort("DAY_OF_WEEK")\
    .withColumnRenamed("count", "NUM_OF_WEEK")\
    .show()

+-----------+-----------+
|DAY_OF_WEEK|NUM_OF_WEEK|
+-----------+-----------+
|          1|     865543|
|          2|     844600|
|          3|     855897|
|          4|     872521|
|          5|     862209|
|          6|     700545|
|          7|     817764|
+-----------+-----------+

CPU times: user 5.35 ms, sys: 106 µs, total: 5.46 ms
Wall time: 5.35 s


#### Thêm trường Hour_AirTime = AIR_TIME / 60

In [9]:
import pyspark.sql.functions as F

Flights = Flights.withColumn('Hour_AirTime', Flights.AIR_TIME/60)
Flights.select("DAY_OF_WEEK", "AIRLINE", "FLIGHT_NUMBER", "Hour_AirTime").show(5)

+-----------+-------+-------------+-----------------+
|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|     Hour_AirTime|
+-----------+-------+-------------+-----------------+
|          4|     AS|           98|2.816666666666667|
|          4|     AA|         2336|4.383333333333334|
|          4|     US|          840|4.433333333333334|
|          4|     AA|          258|              4.3|
|          4|     AS|          135|3.316666666666667|
+-----------+-------+-------------+-----------------+
only showing top 5 rows



#### Có vẻ kết quả số thập phân hơi xấu nhỉ, ta sẽ làm tròn tới 2 chữ số nhé

In [10]:
from pyspark.sql.functions import round, col

Flights = Flights.withColumn('Hour_AirTime', round(col("Hour_AirTime"),2))
Flights.select("DAY_OF_WEEK", "AIRLINE", "FLIGHT_NUMBER", "Hour_AirTime").show(5)

+-----------+-------+-------------+------------+
|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|Hour_AirTime|
+-----------+-------+-------------+------------+
|          4|     AS|           98|        2.82|
|          4|     AA|         2336|        4.38|
|          4|     US|          840|        4.43|
|          4|     AA|          258|         4.3|
|          4|     AS|          135|        3.32|
+-----------+-------+-------------+------------+
only showing top 5 rows



#### Tính tổng số giờ bay của các hãng và sắp xếp theo thứ tứ giảm dần

In [11]:
from pyspark.sql.functions import format_number
Total_Air= Flights.groupBy("AIRLINE")\
        .sum("Hour_AirTime").withColumnRenamed("sum(Hour_AirTime)", "Total_AirTime")\
        .sort("Total_AirTime", ascending=False)\
        .withColumn("Total_AirTime", format_number(col("Total_AirTime"), 0))
Total_Air.show()

+-------+-------------+
|AIRLINE|Total_AirTime|
+-------+-------------+
|     WN|    2,134,872|
|     DL|    1,677,679|
|     AA|    1,661,214|
|     UA|    1,394,956|
|     OO|      727,315|
|     EV|      672,765|
|     B6|      628,286|
|     AS|      451,015|
|     US|      407,138|
|     MQ|      311,223|
|     NK|      254,092|
|     F9|      195,557|
|     VX|      186,429|
|     HA|      107,575|
+-------+-------------+



#### Join bảng Finghts với Total_Air vừa được tạo ra

In [12]:
Airline.show()

+---------+--------------------+
|IATA_CODE|             AIRLINE|
+---------+--------------------+
|       UA|United Air Lines ...|
|       AA|American Airlines...|
|       US|     US Airways Inc.|
|       F9|Frontier Airlines...|
|       B6|     JetBlue Airways|
|       OO|Skywest Airlines ...|
|       AS|Alaska Airlines Inc.|
|       NK|    Spirit Air Lines|
|       WN|Southwest Airline...|
|       DL|Delta Air Lines Inc.|
|       EV|Atlantic Southeas...|
|       HA|Hawaiian Airlines...|
|       MQ|American Eagle Ai...|
|       VX|      Virgin America|
+---------+--------------------+



In [13]:
cond = [Total_Air.AIRLINE == Airline.IATA_CODE]
Total_Air.join(Airline, cond, how='inner').select(Airline.AIRLINE, Total_Air.Total_AirTime).show()

+--------------------+-------------+
|             AIRLINE|Total_AirTime|
+--------------------+-------------+
|United Air Lines ...|    1,394,956|
|    Spirit Air Lines|      254,092|
|American Airlines...|    1,661,214|
|Atlantic Southeas...|      672,765|
|     JetBlue Airways|      628,286|
|Delta Air Lines Inc.|    1,677,679|
|Skywest Airlines ...|      727,315|
|Frontier Airlines...|      195,557|
|     US Airways Inc.|      407,138|
|American Eagle Ai...|      311,223|
|Hawaiian Airlines...|      107,575|
|Alaska Airlines Inc.|      451,015|
|      Virgin America|      186,429|
|Southwest Airline...|    2,134,872|
+--------------------+-------------+



# Thao tac SQL tren Spark.sql


In [14]:
# Them bang bảng vao catalog để sử dụng sql

Flights.createOrReplaceTempView('Flights')
Airline.createOrReplaceTempView('Airlines')
Airport.createOrReplaceTempView('Airports')
my_spark.catalog.listTables()

[Table(name='airlines', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='airports', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='flights', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

#### Tính vận tốc trung bình = DISTANCE / Hour_AirTime

In [15]:
my_spark.sql("""
                SELECT FLIGHT_NUMBER, ORIGIN_AIRPORT, DESTINATION_AIRPORT, TAIL_NUMBER, 
                       ROUND(DISTANCE/Hour_AirTime, 2) AS AVG_SPEED
                FROM Flights

            """).show(5)

+-------------+--------------+-------------------+-----------+---------+
|FLIGHT_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|TAIL_NUMBER|AVG_SPEED|
+-------------+--------------+-------------------+-----------+---------+
|           98|           ANC|                SEA|     N407AS|   513.48|
|         2336|           LAX|                PBI|     N3KUAA|   531.96|
|          840|           SFO|                CLT|     N171US|   518.28|
|          258|           LAX|                MIA|     N3HYAA|   544.65|
|          135|           SEA|                ANC|     N527AS|   436.14|
+-------------+--------------+-------------------+-----------+---------+
only showing top 5 rows



In [16]:
%%time
my_spark.sql("""SELECT DAY_OF_WEEK, count('DAY_OF_WEEK') AS NUM_OF_WEEK 
                FROM Flights 
                GROUP BY DAY_OF_WEEK 
                ORDER BY DAY_OF_WEEK
                """).show()

+-----------+-----------+
|DAY_OF_WEEK|NUM_OF_WEEK|
+-----------+-----------+
|          1|     865543|
|          2|     844600|
|          3|     855897|
|          4|     872521|
|          5|     862209|
|          6|     700545|
|          7|     817764|
+-----------+-----------+

CPU times: user 2.25 ms, sys: 202 µs, total: 2.46 ms
Wall time: 4.22 s


#### Hiển thị các chuyến bay với khoảng cách lớn hơn 1000

In [17]:
my_spark.sql("""SELECT AIRLINE, FLIGHT_NUMBER, ORIGIN_AIRPORT, DESTINATION_AIRPORT, DISTANCE
                FROM Flights
                WHERE DISTANCE>1000
             """)\
        .show(10)

+-------+-------------+--------------+-------------------+--------+
|AIRLINE|FLIGHT_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|DISTANCE|
+-------+-------------+--------------+-------------------+--------+
|     AS|           98|           ANC|                SEA|    1448|
|     AA|         2336|           LAX|                PBI|    2330|
|     US|          840|           SFO|                CLT|    2296|
|     AA|          258|           LAX|                MIA|    2342|
|     AS|          135|           SEA|                ANC|    1448|
|     DL|          806|           SFO|                MSP|    1589|
|     NK|          612|           LAS|                MSP|    1299|
|     US|         2013|           LAX|                CLT|    2125|
|     AA|         1112|           SFO|                DFW|    1464|
|     DL|         1173|           LAS|                ATL|    1747|
+-------+-------------+--------------+-------------------+--------+
only showing top 10 rows



#### Hiển thị các chuyến bay từ của hàng AS cất cánh từ LAX

In [21]:
%%time
my_spark.sql("""
                select AIRLINE, ORIGIN_AIRPORT, DEPARTURE_DELAY, 
                       DEPARTURE_DELAY, SCHEDULED_TIME, AIR_TIME,
                       DISTANCE, WEATHER_DELAY
                from Flights
                WHERE (AIRLINE == 'AS') AND (ORIGIN_AIRPORT == 'LAX')
            """).show(5)

+-------+--------------+---------------+---------------+--------------+--------+--------+-------------+
|AIRLINE|ORIGIN_AIRPORT|DEPARTURE_DELAY|DEPARTURE_DELAY|SCHEDULED_TIME|AIR_TIME|DISTANCE|WEATHER_DELAY|
+-------+--------------+---------------+---------------+--------------+--------+--------+-------------+
|     AS|           LAX|             -5|             -5|           170|     141|     954|         null|
|     AS|           LAX|             -1|             -1|           150|     124|     834|         null|
|     AS|           LAX|             -5|             -5|           170|     139|     954|         null|
|     AS|           LAX|             -1|             -1|           167|     134|     954|         null|
|     AS|           LAX|              3|              3|           145|     120|     834|         null|
+-------+--------------+---------------+---------------+--------------+--------+--------+-------------+
only showing top 5 rows

CPU times: user 1.81 ms, sys: 0 ns, tot

In [2]:
data = my_spark.read.csv('hdfs://localhost:9001/log/data.log', quote=' ')

In [ ]:
data.

In [7]:
my_spark.stop()